# 通过示例学习PyTorch

本教程通过独立的示例介绍 ```<u>PyTorch<\u>``` 的基本概念。

PyTorch 的核心是提供两个主要功能：
> * n 维张量，类似于 NumPy，但可以在 GPU 上运行
> * 用于构建和训练神经网络的自动微分

我们将使用将三阶多项式拟合 ```y = sin(x)``` 的问题作为运行示例。 该网络将具有四个参数，并且将通过使网络输出与实际输出之间的欧几里德距离最小化来进行梯度下降训练，以适应随机数据。

注意

您可以在```本页```浏览各个示例。

## 1. 张量

### Ⅰ.预热：NumPy

在介绍 PyTorch 之前，我们将首先使用 numpy 实现网络。

Numpy 提供了一个 n 维数组对象，以及许多用于操纵这些数组的函数。 Numpy 是用于科学计算的通用框架。 它对计算图，深度学习或梯度一无所知。 但是，通过使用 numpy 操作手动实现网络的前向和后向传递，我们可以轻松地使用 numpy 使三阶多项式适合正弦函数：

In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
# 随机初始化权重
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # 正向传递：计算预测 y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')


99 3267.156712057645
199 2232.6351478785537
299 1528.3182050261917
399 1048.2632717706554
499 720.6895515642644
599 496.9076593771385
699 343.8553352089778
799 239.05704537173096
899 167.21697048762707
999 117.91371822292233
1099 84.03887062103483
1199 60.738232691594746
1299 44.69316361335828
1399 33.63221626564388
1499 25.9989097818173
1599 20.725463614579134
1699 17.07852244400094
1799 14.553846410028086
1899 12.80434305153441
1999 11.590828192638812
Result: y = -0.04840604083464015 + 0.83134880554618 x + 0.008350847523971711 x^2 + -0.0897185928181053 x^3


### Ⅱ.PyTorch：张量

Numpy 是一个很棒的框架，但是它不能利用 GPU 来加速其数值计算。 对于现代深度神经网络，GPU 通常会提供 ```50 倍或更高``` 的加速，因此遗憾的是，numpy 不足以实现现代深度学习。

在这里，我们介绍最基本的 PyTorch 概念：**张量**。 PyTorch 张量在概念上与 numpy 数组相同：张量是 n 维数组，PyTorch 提供了许多在这些张量上进行操作的函数。 在幕后，张量可以跟踪计算图和梯度，但它们也可用作科学计算的通用工具。

与 numpy 不同，PyTorch 张量可以利用 GPU 加速其数字计算。 要在 GPU 上运行 PyTorch 张量，您只需要指定正确的设备即可。

在这里，我们使用 PyTorch 张量将三阶多项式拟合为正弦函数。 像上面的 numpy 示例一样，我们需要手动实现通过网络的正向和反向传递：

In [4]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU()  (取消注释以在 GPU 上运行)

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
# 随机初始化权重
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # 正向传递：计算预测 y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    # 反向螺旋体，用于计算 a、b、c、d 相对于损耗的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    # 使用梯度下降更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 1169.4158935546875
199 783.3199462890625
299 525.9511108398438
399 354.30584716796875
499 239.77244567871094
599 163.30593872070312
699 112.22518920898438
799 78.0818862915039
899 55.2454719543457
999 39.961639404296875
1099 29.72553825378418
1199 22.865009307861328
1299 18.26358413696289
1399 15.174903869628906
1499 13.09998893737793
1599 11.704924583435059
1699 10.766140937805176
1799 10.133827209472656
1899 9.707563400268555
1999 9.419919967651367
Result: y = 0.015452693216502666 + 0.8375528454780579 x + -0.002665845211595297 x^2 + -0.09060106426477432 x^3


### 3.PyTorch：张量 与 Autograd

这里我们准备一个三阶多项式，通过最小化平方欧几里得距离来训练，并预测函数 ```y = sin(x)``` 在 ```-pi``` 到 ```pi``` 上的值。

此实现使用了 PyTorch 张量(tensor)运算来实现前向传播，并使用 PyTorch Autograd 来计算梯度。

PyTorch 张量表示计算图中的一个节点。 如果x是一个张量，且 ```x.requires_grad=True``` ，则 ```x.grad``` 是另一个张量，它保存了x相对于某个标量值的梯度。

In [6]:
import torch
import math

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
# 创建张量来保存输入和输出。
# 默认情况下，requires_grad=False，这表明在向后传递期间，我们不需要计算相对于这些张量的梯度。
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
# 为权重创建随机张量。
# 对于三阶多项式，我们需要 4 个权重：y = a + b x + c x^2 + d x^3 
# 设置 requires_grad=True 表示我们要在向后传递期间计算相对于这些张量的梯度。
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    # 正向传递：使用张量上的运算计算预测的 y。
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    # loss.item（） 获取损失中持有的标量值。
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    # 使用自动渐变来计算向后传递。
    # 此调用将计算相对于所有张量的损失梯度，requires_grad=True。
    # 在此调用a.grad，b.grad之后。
    # c.grad 和 d.grad 将分别是保持相对于 a、b、c、d 的损失梯度的张量。
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # 使用梯度下降手动更新权重。
    # 换行torch.no_grad（），因为权重requires_grad=True，但我们不需要在自动升级中跟踪它。
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        # 更新权重后手动清零渐变
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 1503.9141845703125
199 1011.8036499023438
299 682.1997680664062
399 461.2796325683594
499 313.0946044921875
599 213.62030029296875
699 146.7905731201172
799 101.8546142578125
899 71.61383056640625
999 51.24409866333008
1099 37.51081848144531
1199 28.242835998535156
1299 21.982315063476562
1399 17.74897003173828
1499 14.88352108001709
1599 12.941917419433594
1699 11.624869346618652
1799 10.730536460876465
1899 10.122562408447266
1999 9.708781242370605
Result: y = -0.022038761526346207 + 0.8359349966049194 x + 0.0038020508363842964 x^2 + -0.09037093073129654 x^3


## PyTorch：定义新的 Autograd 函数

这里我们准备一个三阶多项式，通过最小化平方欧几里得距离来训练，并预测函数<font color="orange"> $y = sin(x)$ </font>在<font color="orange"> $-pi$ </font>到<font color="orange"> $pi$ </font>上的值。

这里我们不将多项式写为<font color="orange"> $y = a + bx + cx^2 + dx^3$ </font>，而是将多项式写为<font color="orange"> $y = a + bP_3(c + dx)$ </font>，其中<font color="orange"> $P_3(x) = 1/2 (5x ^ 3 - 3x)$ </font>是```三次勒让德多项式。```

此实现使用了 PyTorch 张量(tensor)运算来实现前向传播，并使用 PyTorch Autograd 来计算梯度。

在此实现中，我们实现了自己的自定义 Autograd 函数来执行<font color="orange"> $P'_3(x)$ </font>。 从数学定义上讲，<font color="orange">$ P'_3(x) = 3/2 (5x ^ 2 - 1)$</font> ：

In [7]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.

    我们可以通过子类化 torch.autograd.Function 
    并实现在 Tensors 上运行的向前和向后传递
    来实现我们自己的自定义 autograd 函数。
    """

    # python staticmethod 返回函数的静态方法
    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.

        在正向传递中，我们接收到包含输入的张量，并返回包含输出的张量。
        ctx 是一个上下文对象，可用于隐藏信息以进行向后计算。
        您可以使用ctx.save_for_backward方法缓存任意对象，以便在向后传递中使用。
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.

        在反向传递中，我们收到一个包含相对于输出的损耗梯度的张量，并且我们需要计算相对于输入的损耗梯度。
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
# 创建张量来保存输入和输出。
# 默认情况下，requires_grad=False，
# 这表明在向后传递期间，我们不需要计算相对于这些张量的梯度。
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
# 为权重创建随机张量。
# 对于此示例，我们需要 4 个权重：y = a + b * P3（c + d * x），这些权重需要在距离正确结果不太远的地方进行初始化，以确保收敛。
# 设置 requires_grad=True 表示我们希望在向后传递期间计算相对于这些张量的梯度。
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    # 为了应用我们的函数，我们使用Function.apply方法。我们将其别名为"P3"。
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    # 正向传递：使用运算计算预测的y;
    #         我们使用自定义自动升级操作计算 P3。
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')


99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03520202636719
499 50.978515625
599 37.40313720703125
699 28.20686912536621
799 21.973186492919922
899 17.745729446411133
999 14.877889633178711
1099 12.931766510009766
1199 11.610918998718262
1299 10.714248657226562
1399 10.105474472045898
1499 9.692106246948242
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = -1.765793067320942e-11 + -2.208526849746704 * P3(9.924167737596079e-11 + 0.2554861009120941 x)
